In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
import os
import time
import math
import webbrowser
import numpy as np
import re
import sys
from pandas import DataFrame

In [2]:
chrome_path = "C:\Temp\chromedriver.exe"
options = webdriver.ChromeOptions()
#지정한 user-agent로 설정합니다.
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
options.add_argument("no-sandbox")
options.add_argument("disable-gpu")
options.add_argument("--lang=ko_KR")
options.add_argument('user-agent=' + user_agent)
driver = webdriver.Chrome(chrome_path,options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", 
{"source": """ Object.defineProperty(navigator, 'webdriver', { get: () => undefined }) """})
url = "https://www.oliveyoung.co.kr/store/main/main.do?oy=0"
driver.get(url)
driver.set_window_size(1920,1080)
driver.maximize_window()

C:\Users\won\AppData\Local\Temp\ipykernel_2272\1087843084.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path,options=options)


In [3]:
prod_text = '선케어'
driver.find_element(By.CLASS_NAME,"placeholder_area").click()
prod = driver.find_element(By.ID,"query")
prod.send_keys(prod_text)
time.sleep(1)
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "searchSubmit"))).click()

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [4]:
#선케어 제품들  url 가져오기
suncare_url_list = []
items = soup.find_all("div",attrs={"class":"prd_info"})
for item in items:
    url=item.find('a')['href']
    suncare_url_list.append(url)


In [5]:
driver.get(suncare_url_list[0])
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "reviewInfo"))).click()
time.sleep(2)


In [6]:
html_type = driver.page_source
soup_type = BeautifulSoup(html_type,'html.parser')
time.sleep(2)
#상품이름 가져오기
pos_prod_name_list=[]
pos_prod_name=soup_type.find('p',attrs={'class':'prd_name'}).get_text()


#페이지 넘기기면서 데이터 가져오기(리뷰,평점,아이디)
pos_num=1
pos_user_name_list=[]
pos_rate_list=[]
pos_review_list=[]
# postive_reviews=['//*[@id="searchPoint"]/li[2]','//*[@id="searchPoint"]/li[3]']
# negative_reviws=['//*[@id="searchPoint"]/li[2]','//*[@id="searchPoint"]/li[2]']
#체험단 제거
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID,'searchType_1'))).click()
time.sleep(1)
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID,'searchType_3'))).click()
time.sleep(1)
#리뷰 검색 필터
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME,'btnOption'))).click()
#5점
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[2]'))).click()
time.sleep(2)
#4점

WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[3]'))).click()    
time.sleep(2)
#확인 버튼
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'btnFilterConfirm'))).click()
time.sleep(2)

html_type = driver.page_source
soup_type = BeautifulSoup(html_type,'html.parser')
time.sleep(2)

In [7]:
for num in range(1,11):
    #사용자이름

    try:
        user_names=soup_type.find_all('a',attrs={'class':'id'})
        for user_name in user_names:
            pos_user_name_list.append(user_name.get_text())
    except:        
        pos_user_name_list.append("err")

    #평점
    try:
        non_rates=soup_type.find_all('div',attrs={'class':'score_area'})
        for rates in non_rates:
            rate = rates.find('span',attrs={'class':'point'}).get_text()
            rate = re.findall(r'\d', rate)
            pos_rate_list.append(rate[1])

    except:
        pos_rate_list.append("rate_err")

    #리뷰
    try:    
        reviews=soup_type.find_all('div',attrs={'class':'txt_inner'})
        for review in reviews:
            review = re.compile('[가-힣]+').findall(review.get_text())
            pos_review_list.append(review)
    except:
        pos_review_list.append("review_err")
    try:
        
        page_element=soup_type.find('div',attrs={'class':'pageing'})
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, "{}".format(num+1)))).click()
        time.sleep(1)
        html_type = driver.page_source
        soup_type = BeautifulSoup(html_type,'html.parser')
        time.sleep(2)
    except:
        break

In [8]:
#리뷰 필터 버튼
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME,'btnOption'))).click()

#초기화 버튼
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'resetBtn'))).click()

#팝업창 확인 버튼
from selenium.webdriver.common.alert import Alert
da = Alert(driver)
da.accept()

In [9]:

#리뷰 검색 필터
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME,'btnOption'))).click()
time.sleep(1)
#2점
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[5]'))).click()
time.sleep(1)
#1점
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[6]'))).click()
time.sleep(1)
#확인 버튼
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'btnFilterConfirm'))).click()
time.sleep(1)

In [10]:
html_type = driver.page_source
soup_type = BeautifulSoup(html_type,'html.parser')
time.sleep(1)
neg_num=1
neg_user_name_list=[]
neg_rate_list=[]
neg_review_list=[]

#상품이름 가져오기
neg_prod_name_list=[]
neg_prod_name=soup_type.find('p',attrs={'class':'prd_name'}).get_text()


for num in range(1,11):#잠시 2페이지 까지
    #사용자이름

    try:
        user_names=soup_type.find_all('a',attrs={'class':'id'})
        for user_name in user_names:
            neg_user_name_list.append(user_name.get_text())
    except:        
        neg_user_name_list.append("err")

    #평점
    try:
        non_rates=soup_type.find_all('div',attrs={'class':'score_area'})
        for rates in non_rates:
            rate = rates.find('span',attrs={'class':'point'}).get_text()
            rate = re.findall(r'\d', rate)
            neg_rate_list.append(rate[1])

    except:
        neg_rate_list.append("rate_err")

    #리뷰
    try:    
        reviews=soup_type.find_all('div',attrs={'class':'txt_inner'})
        for review in reviews:
            review = re.compile('[가-힣]+').findall(review.get_text())
            neg_review_list.append(review)
    except:
        neg_review_list.append("review_err")
    try:    
        page_element=soup_type.find('div',attrs={'class':'pageing'})
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, "{}".format(num+1)))).click()
        time.sleep(1)
        html_type = driver.page_source
        soup_type = BeautifulSoup(html_type,'html.parser')
        time.sleep(2)
    except:
        break

#5점
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[2]'))).click()
#4점
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[3]'))).click()
#2점
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[5]'))).click()
#1점
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[6]'))).click()
#확인 버튼
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'btnFilterConfirm'))).click()
#초기화 버튼
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'resetBtn'))).click()

WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'btnFilterConfirm'))).click()

#리뷰에서 화장품 특성들 가져오는 코드
html_type = driver.page_source
soup_type = BeautifulSoup(html,'html.parser')

infos = soup_type.find_all('dl',attrs={'class':'poll_type2 type3'})


for info in infos:
    tag = info.find('dt')
    
    print(tag.get_text())
    rates_1=info.find_all('span',attrs={'class':'txt'})
    rates_2=info.find_all('em',attrs={'class':'per'})
    for rate1,rate2 in zip(rates_1,rates_2):
        print(rate1.get_text(),rate2.get_text(),'\n')

        

### 리뷰 가져오기

In [11]:
# column=['상품 이름','유저 ID','유저평점','리뷰']
# df = pd.DataFrame([pos_prod_name],[pos_user_name_list],[pos_rate_list],[pos_review_list],columns=column)

['제가',
 '지금',
 '통째',
 '사는',
 '유일한',
 '선크림',
 '입니당가격대비',
 '이만한게',
 '없어요',
 '저희',
 '아빠가',
 '선크림',
 '떨어지면',
 '꼭',
 '이거',
 '사오라고',
 '하세요',
 '아버지가',
 '대이신데',
 '썬크림을',
 '꼭',
 '바르고',
 '다니시거든요',
 '근데',
 '텁텁하고',
 '무겁고',
 '백탁있으면',
 '답답하다고',
 '못바르겠다그러는데이건',
 '그냥',
 '로션바른거',
 '같다고',
 '듬뿍바르고',
 '다니십니다',
 '평생사줄테니까',
 '오래오래',
 '건강하셨으면',
 '좋겠네요',
 '진짜',
 '다른',
 '선크림들하고',
 '다르게',
 '물처럼',
 '잘',
 '발리고',
 '끈적임',
 '하나없어서',
 '너무',
 '좋아요',
 '이거',
 '입덕하고',
 '선크림은',
 '식물나라',
 '제품만',
 '써요',
 '진짜',
 '식물나라',
 '선크림이',
 '이거',
 '말고도',
 '많은데',
 '다',
 '좋은거',
 '같아요',
 '근데',
 '이',
 '선크림이',
 '발림성도',
 '좋고',
 '개씩이나',
 '들어있고',
 '제형이',
 '저랑',
 '잘',
 '맞아요',
 '진짜',
 '다들',
 '선크림',
 '이거',
 '꼭쓰세요',
 '제',
 '주변인들',
 '다',
 '이선크림',
 '입덕했어요',
 '한여름',
 '최고의',
 '선크림싸이닉',
 '유기자차',
 '잘',
 '쓰다가',
 '점점',
 '습해져서',
 '좀더',
 '가벼운',
 '걸',
 '찾다가',
 '얘로',
 '정착싸이닉',
 '자작나무랑',
 '필터구성',
 '정말',
 '비슷함사용감과',
 '백탁',
 '가장',
 '가벼움',
 '바디에',
 '써도',
 '산뜻해서',
 '쾌적한',
 '자차생활',
 '가능',
 '기획구성',
 '너무',
 '선녀임',
 '올',
 '여름은',
 '너랑',
 '함께다',
 '너무',
 '좋아요',
 '이거는',
 

review_list=[]
review_num=1
reviews = soup_type.find_all('div',attrs={'class':'txt_inner'})
for review in reviews:
        
    review_list.append(review.get_text())
    
    if(review_num %10 == 0):

In [12]:
pos_data={"상품이름" : pos_prod_name,
         "유저 ID" : pos_user_name_list,
          "유저평점" : pos_rate_list,
          '리뷰':pos_review_list
          
         }

pos_df = pd.DataFrame(pos_data)
pos_df

ValueError: All arrays must be of the same length

In [ ]:
neg_data={"상품이름" : neg_prod_name,
         "유저 ID" : neg_user_name_list,
          "유저평점" : neg_rate_list,
          '리뷰':neg_review_list
          
         }

neg_df = pd.DataFrame(neg_data)
neg_df

In [ ]:
#시간 활용 파일 저장
import datetime
suffix = datetime.datetime.now().strftime('%y%m%d_%H%M%S')
pos_fileName = suffix + 'pos_sunscreen.csv'
neg_fileName = suffix + 'neg_sunscreen.csv'

In [ ]:
os.chdir('C:\Temp')
pos_df.to_csv(pos_fileName, index=False, encoding='cp949')

In [ ]:
os.chdir('C:\Temp')
neg_df.to_csv(neg_fileName, index=False, encoding='cp949')


fc_name = sunscreen_reviews.csv
df = pd.DataFrame()
df['ranking_no'] = pd.Series(ranking_no_list)

df

df.to_csv(fc_name,index='ranking_no',encoding='cp949')